# Video of brain spinning with all clusters, organized by insertion

In [1]:
from brainwidemap import bwm_query, bwm_units
from one.api import ONE

one = ONE(base_url='https://openalyx.internationalbrainlab.org')
# Dataframe with info on all sessions and probes released for the BWM
bwm_df = bwm_query(one)
# Dataframe with information on all neurons used in the analyses in the BWM paper
unit_df = bwm_units(one)

c:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\__init__.py:202: DeprecationWarning: ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use this module instead
  warnings.warn('ibllib.atlas is deprecated. Please install iblatlas using "pip install iblatlas" and use '


Loading bwm_query results from fixtures/2023_12_bwm_release.csv
Loading bwm_query results from fixtures/2023_12_bwm_release.csv


c:\Users\Dan\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\atlas.py:13: DeprecationWarning: ibllib.atlas.regions.BrainRegions is deprecated. Use iblatlas.regions.BrainRegions instead
  warnings.warn(warning_text, DeprecationWarning)


In [ ]:
# Use this code instead to load ephys atlas data
# https://github.com/int-brain-lab/paper-ephys-atlas/blob/main/sources/examples/00_load_clusters_tables.py

In [2]:
from iblatlas import atlas
aa = atlas.AllenAtlas()
from iblatlas.regions import BrainRegions
br = BrainRegions()
import numpy as np
import pandas as pd
import time
import seaborn as sns

In [3]:
bwm_df.head()

,pid,eid,probe_name,session_number,date,subject,lab
0,56f2a378-78d2-4132-b3c8-8c1ba82be598,6713a4a7-faed-4df2-acab-ee4e63326f8d,probe00,1,2020-02-18,NYU-11,angelakilab
1,47be9ae4-290f-46ab-b047-952bc3a1a509,56956777-dca5-468c-87cb-78150432cc57,probe01,1,2020-02-21,NYU-11,angelakilab
2,6be21156-33b0-4f70-9a0f-65b3e3cd6d4a,56956777-dca5-468c-87cb-78150432cc57,probe00,1,2020-02-21,NYU-11,angelakilab
3,8dfb86c8-d45c-46c4-90ec-33078014d434,4364a246-f8d7-4ce7-ba23-a098104b96e4,probe01,1,2020-01-20,NYU-12,angelakilab
4,c893c0a3-5597-49cf-baa1-60efdfdef542,b182b754-3c3e-4942-8144-6ee790926b58,probe01,1,2020-01-21,NYU-12,angelakilab


In [4]:
unit_df.head()

,Beryl,eid,channels,depths,uuids,cluster_id,amp_max,amp_min,amp_median,amp_std_dB,...,label,x,y,z,acronym,atlas_id,axial_um,lateral_um,pid,ks2_label
0,ACAd,0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde,306,3080.0,9ce7d244-4cfa-429c-a1bd-6ea7d771f87b,310,0.000137,0.000044,0.000080,1.120884,...,1.0,-0.000988,0.001474,-0.002444,ACAd6a,919,3080.0,59.0,1ab86a7f-578b-4a46-9c9c-df3be97abcca,None
1,ACAd,0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde,307,3080.0,c51feba4-3b4e-4999-aef6-f1f50f903af6,311,0.000137,0.000039,0.000073,1.505985,...,1.0,-0.000988,0.001474,-0.002444,ACAd6a,919,3080.0,27.0,1ab86a7f-578b-4a46-9c9c-df3be97abcca,None
2,ACAd,0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde,308,3100.0,1c585cde-5dbd-48cb-b446-1df386b87f2c,313,0.000200,0.000071,0.000126,1.146507,...,1.0,-0.000988,0.001474,-0.002424,ACAd6a,919,3100.0,43.0,1ab86a7f-578b-4a46-9c9c-df3be97abcca,None
3,ACAd,0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde,311,3120.0,4f637be0-a5ff-4867-84a4-66bbbc7264e0,320,0.000111,0.000052,0.000070,1.296398,...,1.0,-0.000988,0.001476,-0.002404,ACAd6a,919,3120.0,27.0,1ab86a7f-578b-4a46-9c9c-df3be97abcca,None
4,ACAd,0f77ca5d-73c2-45bd-aa4c-4c5ed275dbde,313,3140.0,abef29c9-ca03-4cb1-902c-089bec2060f7,323,0.000139,0.000055,0.000082,1.176087,...,1.0,-0.000988,0.001480,-0.002384,ACAd6a,919,3140.0,11.0,1ab86a7f-578b-4a46-9c9c-df3be97abcca,None


In [5]:
def hex_to_rgb(hex_color):
    # Remove '#' if present
    if hex_color.startswith('#'):
        hex_color = hex_color[1:]

    # Convert hex to RGB
    red = int(hex_color[0:2], 16) / 255.0
    green = int(hex_color[2:4], 16) / 255.0
    blue = int(hex_color[4:6], 16) / 255.0

    return red, green, blue

In [6]:
ulabs = np.unique(bwm_df.lab)

In [7]:
lab_data = []

id2color = {}

for lab in ulabs:
    pids = bwm_df.pid[bwm_df.lab == lab]
    
    # print((lab, len(pids)))


    coords = []

    for pid in pids:
        units = unit_df[unit_df.pid==pid]
        # print(len(units))
        for i, unit in units.iterrows():
            coord = aa.xyz2ccf([unit.x, unit.y, unit.z])
            if unit.atlas_id in id2color.keys():
                color = id2color[unit.atlas_id]
            else:
                color = hex_to_rgb(aa.regions.hexcolor[aa.regions.id2index(unit.atlas_id)[1][0][0]])
                id2color[unit.atlas_id] = color
            coords.append([coord, color])

    lab_df = pd.DataFrame(columns=['mlapdv', 'color'], data=coords)

    lab_data.append(lab_df)

In [8]:
institution_map = {'cortexlab': 'UCL', 'mainenlab': 'CCU', 'zadorlab': 'CSHL (Z)',
                       'churchlandlab': 'CSHL (C)', 'angelakilab': 'NYU',
                       'wittenlab': 'Princeton', 'hoferlab': 'SWC', 'mrsicflogellab': 'SWC',
                       'danlab': 'Berkeley', 'steinmetzlab': 'UW', 'churchlandlab_ucla': 'UCLA', 'hausserlab':'UCL'}
colors = np.concatenate([sns.color_palette("Dark2"), sns.color_palette('Set2')[0:2]])
institutions = ['UCL', 'CCU', 'CSHL (C)', 'NYU', 'Princeton', 'SWC', 'Berkeley', 'CSHL (Z)',
                'UW', 'UCLA']
institution_colors = {}
for i, inst in enumerate(institutions):
    institution_colors[inst] = colors[i]

In [9]:
ulabs

array(['angelakilab', 'churchlandlab', 'churchlandlab_ucla', 'cortexlab',
       'danlab', 'hausserlab', 'hoferlab', 'mainenlab', 'mrsicflogellab',
       'steinmetzlab', 'wittenlab', 'zadorlab'], dtype=object)

In [10]:
names_clean = [institution_map[lab] for lab in ulabs]

In [11]:
lab_colors = [institution_colors[institution_map[lab]] for lab in ulabs]

In [12]:
lab_colors = np.concatenate([sns.color_palette("Dark2"), sns.color_palette('Set2')])

In [27]:
def tick(start):
    t = time.time() - start
    urchin.camera.set_brain_rotation(t * 5)
    time.sleep(1/30)

In [13]:
import oursin as urchin
urchin.setup()

(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


In [34]:
# Load roo
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [35]:
urchin.ccf25.root.set_visibility(True, side=urchin.utils.Side.FULL)
urchin.ccf25.root.set_color([0,0,0])
urchin.ccf25.root.set_material('transparent-lit')
urchin.ccf25.root.set_alpha(0.15)

In [36]:
urchin.camera.main.set_rotation([-45, 0, 0])
urchin.camera.set_brain_rotation(0)
urchin.camera.main.set_mode('perspective')

In [37]:
text = urchin.text.Text()
text.set_position([0.25, -0.8])
text.set_color('#000000')
text.set_font_size(72)

In [38]:
all_meshes = []
all_colors = []

start = time.time()

for i, lab_df in enumerate(lab_data):

    positions = []
    colors = []
    scales = []

    ccolor = list(lab_colors[i])
    
    text.set_text(names_clean[i])
    text.set_color(urchin.utils.rgb_to_hex((int(ccolor[0]*255), int(ccolor[1]*255), int(ccolor[2]*255))))

    lab_meshes = []

    for j, row in lab_df.iterrows():
        positions.append([row.mlapdv[1], row.mlapdv[0], row.mlapdv[2]])
        colors.append(ccolor)
        scales.append([0.03, 0.03, 0.03])

        all_colors.append(row.color)

        if len(positions) == 50:
            meshes = urchin.meshes.create(50)
            urchin.meshes.set_positions(meshes, positions)
            urchin.meshes.set_colors(meshes, colors)
            urchin.meshes.set_scales(meshes, scales)

            all_meshes += meshes
            lab_meshes += meshes

            positions = []
            colors = []
            scales = []

            tick(start)

    for i in np.arange(30):
        tick(start)
    urchin.meshes.set_colors(lab_meshes, [[0,0,0]]*len(lab_meshes))
    urchin.meshes.set_scales(lab_meshes, [[0.01,0.01,0.01]]*len(lab_meshes))
    
    for i in np.arange(6):
        tick(start)
    
urchin.meshes.set_colors(all_meshes, all_colors)
urchin.meshes.set_scales(all_meshes, [[0.02,0.02,0.02]]*len(all_meshes))
text.set_position([2,2])

(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


...

(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: efe446f4, copy this ID into the renderer to connect.


In [17]:
urchin.camera.main.set_rotation([-45, 0, 0])
for i in np.arange(0,500):
    urchin.camera.set_brain_rotation(i)
    time.sleep(1/30)